# 如何使用 MultiQueryRetriever

基于距离的[向量数据库](/docs/concepts/vectorstores/)检索通过[嵌入](/docs/concepts/embedding_models/)（表示）查询到高维空间，并根据距离度量找到相似的嵌入文档。但是，检索可能会因查询措辞的细微变化或嵌入未能很好地捕捉数据语义而产生不同的结果。提示工程/调优有时会手动解决这些问题，但可能很繁琐。

[MultiQueryRetriever](https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html)通过使用LLM为给定的用户输入查询生成多个不同视角的查询，自动化了提示调优过程。对于每个查询，它检索一组相关文档，并对所有查询取唯一并集，以获得更大范围的潜在相关文档集。通过对同一问题生成多个视角，`MultiQueryRetriever`可以缓解基于距离的检索的一些限制，并获得更丰富的结果集。

让我们使用[RAG教程](/docs/tutorials/rag)中的[Lilian Weng的LLM驱动自主代理](https://lilianweng.github.io/posts/2023-06-23-agent/)博客文章来构建一个向量存储：

In [ ]:
# 构建一个示例向量数据库
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 加载博客文章
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# 分割
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# 向量数据库
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

USER_AGENT environment variable not set, consider setting it to identify your requests.


#### 简单用法

指定用于查询生成的LLM，检索器将完成其余工作。

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "任务分解的方法有哪些？"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [ ]:
# 为查询设置日志记录
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [4]:
unique_docs = retriever_from_llm.invoke(question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How can Task Decomposition be achieved through different methods?', '2. What strategies are commonly used for Task Decomposition?', '3. What are the various ways to break down tasks in Task Decomposition?']


5

请注意，由[检索器](/docs/concepts/retrievers/)生成的底层查询会以`INFO`级别记录。

#### 提供自定义提示

在底层，`MultiQueryRetriever`使用特定的[提示](https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html)生成查询。要自定义此提示：

1. 创建一个带有问题输入变量的[PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html)；
2. 实现一个类似于下面的[输出解析器](/docs/concepts/output_parsers)，将结果拆分为查询列表。

提示和输出解析器必须支持生成查询列表。

In [ ]:
from typing import List

from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


# 输出解析器将LLM结果拆分为查询列表
class LineListOutputParser(BaseOutputParser[List[str]]):
    """用于行列表的输出解析器。"""

    def parse(self, text: str) -> List[str]:
        lines = text.strip().split("\n")
        return list(filter(None, lines))  # 移除空行


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""您是一个AI语言模型助手。您的任务是生成五个
    不同版本的用户问题，以从向量数据库中检索相关文档。通过对用户问题生成多个视角，
    您的目标是帮助用户克服基于距离的相似性搜索的一些限制。
    提供这些替代问题，并用换行符分隔。
    原始问题：{question}""",
)
llm = ChatOpenAI(temperature=0)

# 链
llm_chain = QUERY_PROMPT | llm | output_parser

# 其他输入
question = "任务分解的方法有哪些？"

In [ ]:
# 运行
retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines"是解析输出的键（属性名称）

# 结果
unique_docs = retriever.invoke("课程中关于回归的内容是什么？")
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide insights on regression from the course material?', '2. How is regression discussed in the course content?', '3. What information does the course offer regarding regression?', '4. In what way is regression covered in the course?', "5. What are the course's teachings on regression?"]


9